
MIT Open Source License: <br>
Copyright (c) 2018 Daniel C. Elton 

<sub>Permission is hereby granted, free of charge, to any person obtaining a copy</sub>
<sub>of this software and associated documentation files (the "Software"), to deal</sub>
<sub>in the Software without restriction, including without limitation the rights</sub>
<sub>to use, copy, modify, merge, publish, distribute, sublicense, and/or sell</sub>
<sub>copies of the Software, and to permit persons to whom the Software is</sub>
<sub>furnished to do so, subject to the following conditions:</sub>

<sub>The above copyright notice and this permission notice shall be included in all</sub>
<sub>copies or substantial portions of the Software.</sub>

<sub>THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR</sub>
<sub>IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,</sub>
<sub>FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE</sub>
<sub>AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER</sub>
<sub>LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,</sub>
<sub>OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE</sub>
<sub>SOFTWARE.</sub>

# Generate and save coordinates

In [ ]:
from openbabel import OBConversion
import rdkit.Chem as Chem
from rdkit.Chem.rdForceFieldHelpers import *
from rdkit.Chem.rdDistGeom import EmbedMolecule, EmbedMultipleConfs
import openbabel as ob
#import pybel as pybel
import pandas as pd
import numpy as np
#from ccdc import io


#-------- Read the data -----------------------------
data = pd.read_excel('../datasets/Huang_Massa_data_with_all_SMILES.xlsx', skipfooter=1)
#data = pd.read_excel('datasets/sensitivity_database.xlsx')
data['Mols'] = data['SMILES'].apply(Chem.MolFromSmiles)
#print(data.columns.values)

make_combined_file = True
check_CCSD = True

if make_combined_file: file_combined = open("HM_combined.txt", 'w')

    
#---------------------------------------------
def gen_coords_rdkit(mol):
    ''' takes a Rdkit Mol object with no coords and adds coordinates to it'''
       
    Nconfs = mol.GetNumConformers()
    if (Nconfs > 0):
        print("WARNING: there is already ", Nconfs, "conformers in molecule")
        
    #-----  generate initial coord embedding
    err = EmbedMolecule(mol, clearConfs=True)
    
    #--- Attempt to generate multiple with RDKit confs and find lowest energy one 
    #err = EmbedMultipleConfs(mol, numConfs=100)
    #energies = []
    #for conf in mol.GetConformers():
    #    iD = conf.GetId()
    #    mmff_props = MMFFGetMoleculeProperties(mol, mmffVariant='MMFF94')
    #    ff = MMFFGetMoleculeForceField(mol, mmff_props, conf_id=iD)
    #    energy = ff.CalcEnergy()
    #    energies.append(energy)
    #energies = np.asarray(energies, dtype=float)
    #minenergy = min(energies)
    #min_id = energies.index(minenergy)
    bad = False

    if (err == -1):
        err = EmbedMolecule(mol, useRandomCoords=False)
        if (err == -1):
            print("ERROR: could not create intial coordinate embedding")
            bad = True
            
    #---- optimize coords with RDkit --------
    if (bad != True):
        err = MMFFOptimizeMolecule(mol,confId=-1, mmffVariant='MMFF94', maxIters=10000, nonBondedThresh=200.0)
        if (err == -1):
             print("ERROR: forcefield could not be set up")
        if (err == 1): 
             print("WARNING: more iterations are required for convergence.")
    
    Nconfs = mol.GetNumConformers()
    if (Nconfs > 1):
        print("WARNING: mol", i, "has ", Nconfs, "conformers in the mol file instead of just one")
        
    if( mol.GetNumHeavyAtoms() < 2):
        print("WARNING only", mol.GetNumHeavyAtoms(), "heavy atoms")
    
    return Chem.MolToMolBlock(mol)

#----------------------------------------------------
def check_CCSD_for_mol(refcode):
    ## as of 2017 this only works in Python 2 =(
    csd_reader = io.EntryReader('CSD')

    try:
        molCSD = csd_reader.crystal(refcode)
    except (RuntimeError):
        return 'ERROR'

    return molCSD.to_string('mol2')

#----------------------------------------------------
def check_CCSD_dir_for_mol(refcode, directory='HM_all_mol/'):
    """ looks in a directory for a file with file name refcode.mol2 and then reads it to string
        if it exists. Otherwise, it returns a string with the value "ERROR" 
    """
    try:
        with open(directory+refcode+'.mol') as file:
            return file.read()
    except:
        print("Error finding molecule in CCSD directory")
        return "ERROR"

#------- main loop ----------------------------
def gen_coords(mol_list, verbose=True, timing=True):
    for (i, mol) in enumerate(mol_list):
        print(i)

        mol = Chem.AddHs(mol) #important to make sure nitro group representation is double bonds here, not charges

        NumAtoms = mol.GetNumAtoms()

        if (check_CCSD):
            molstring = check_CCSD_dir_for_mol(data['Molecular Name'][i])
            if (molstring == 'ERROR'):
                molstring = gen_coords_rdkit(mol)
        else:
            molstring = gen_coords_rdkit(mol)

        #obmol = pybel.readstring("mol", molstring)

        #------- create open Babel mol object from molstring
        obConversion = ob.OBConversion()
        obConversion.SetInAndOutFormats("mol", "xyz")
        obmol = ob.OBMol()
        obConversion.ReadString(obmol, molstring)

        #----- solve problem of multiple disconnected molecules in CCSD mol2 files
        NumResidues = obmol.NumResidues()

        if (NumResidues > 1):
            print("WARNING: Multiple Residues detected")
            for i in range(1,NumResidues):
                obmol.DeleteResidue(obmol.GetResidue(i))

        NumOBatoms = obmol.NumAtoms()
        if (NumAtoms != NumOBatoms):
            print("Differing number of atoms,", NumAtoms, "vs", NumOBatoms)

        for i in range(NumAtoms+1, NumOBatoms):
            obmol.DeleteAtom(i)

        Nconfs = obmol.NumConformers()
        if (Nconfs > 1):
            print("WARNING: mol", i, "has ", Nconfs, "conformers in the mol file instead of just one")


        #----- conformation search in Open Babel 
        ff = ob.OBForceField.FindForceField("mmff94")
        ff.Setup(obmol)

        #ff.RandomRotorSearch(50,10)
        ff.FastRotorSearch()
        ff.WeightedRotorSearch(12000,60)

        #conformers 	The number of random conformers to consider during the search.
        #geomSteps 	The number of steps to take during geometry optimization for each conformer.

        ff.ConjugateGradients(1000, 1e-5) #n_step, e_convergence

        #----- write .xyz file    
        fileheader = str(i)
        obConversion.WriteFile(obmol, fileheader+".xyz")

        fileheader = str(data['Molecular Name'][i])

        obConversion.WriteFile(obmol, fileheader+".xyz")

        obConversion.SetInAndOutFormats("mol", "mol")

        obConversion.WriteFile(obmol, fileheader+".mol")



        #------ make combined file 
        if make_combined_file: 
            file = open(fileheader+".xyz", 'r')
            text = file.readlines()
            file.close()

            for (j, line) in enumerate(text):
                if ((line == '\n') & (j !=1)):
                    print("WARNING: unexpected blank line at line #",  j, line)
                if (j == 1):
                    #EE = str(data["detonation velocity"].iloc[i])  
                    #EE = str(data["Explosive energy (kj/cc)"].iloc[i])  
                    #EE = ''
                    #file.write(EE+'\n')
                    file_combined.write('\n')
                #elif (j == 0):
                #    pass
                else:
                    file_combined.write(line)
                    #file.write(line)

if ( __name__ == '__main__'):
    gen_coords(list(data['Mols']))

0
